In [44]:

from config_mappings import ConfigMappings
import pandas as pd
from rank import load_config



In [45]:
# Reload the module to get the latest changes
import importlib
import config_mappings
importlib.reload(config_mappings)

# Recreate the ConfigMappings instance with reloaded module
from config_mappings import load_mappings_from_config
m = load_mappings_from_config()
print("✅ Module reloaded successfully")

✅ Module reloaded successfully


In [46]:
# Load configuration from YAML file
config = load_config("rank-config.yaml")
m = ConfigMappings(config)

In [47]:
df_results = pd.read_csv("data/local/stock_evaluation_results.csv", index_col=0)
selected_ticker ='AAK'

In [48]:
df_results['ROIC_long_trend_ratioValue']

ARION-SDB         NaN
AZA               NaN
SHB-A             NaN
SHB-B             NaN
NDA-SE            NaN
               ...   
MAVEN       -0.276837
TALK        -0.097363
TESSIN       0.100699
WAYS        -0.155071
WYLD              NaN
Name: ROIC_long_trend_ratioValue, Length: 702, dtype: float64

In [49]:
selected_ticker = ['AAK','ABB','SAND','VOLV-B']

In [50]:
df_test=df_results[['Sektor','ttm_momentum_clusterRank','ttm_momentum_clusterRank_Sektor_avg','pct_ch_3_m','pct_ch_3_m_Sektor_avg']].loc[selected_ticker]

In [51]:
def prepare_sector_comparison(df_test):
    df_sector_avg = df_test[['Sektor','ttm_momentum_clusterRank_Sektor_avg', 'pct_ch_3_m_Sektor_avg']].drop_duplicates()
    df_sector_avg['is_sektor'] = True
    df_sector_avg.rename(columns={
        'ttm_momentum_clusterRank_Sektor_avg': 'ttm_momentum_clusterRank',
        'pct_ch_3_m_Sektor_avg': 'pct_ch_3_m'
    }, inplace=True)
    df_sector_avg.set_index('Sektor', inplace=True, drop=False)
    df_test = df_test.drop(columns=['ttm_momentum_clusterRank_Sektor_avg', 'pct_ch_3_m_Sektor_avg'])
    df_test['is_sektor'] = False
    return pd.concat([df_test, df_sector_avg], axis=0)

In [52]:
df_final = prepare_sector_comparison(df_test)
df_final

,Sektor,ttm_momentum_clusterRank,pct_ch_3_m,is_sektor
AAK,Dagligvaror,77.1,5.942948,False
ABB,Industri,79.7,11.760991,False
SAND,Industri,74.2,20.686771,False
VOLV-B,Industri,94.8,-8.312877,False
Dagligvaror,Dagligvaror,54.8,-6.787114,True
Industri,Industri,52.2,1.297974,True


In [53]:
# pairs to connect in Parallel coordinate plot
parallel_coordinates_pairs = [
    ('ttm_momentum_clusterRank', 'pct_ch_3_m'),
    ('ttm_momentum_clusterRank_Sektor_avg', 'pct_ch_3_m_Sektor_avg')
]
def plotly_parallel_coordinates(df, pairs):
    import plotly.express as px
    for col1, col2 in pairs:
        if col1 in df.columns and col2 in df.columns:
            fig = px.parallel_coordinates(df, dimensions=[col1, col2], color=col1)
            fig.show()

In [54]:
plotly_parallel_coordinates(df_final, parallel_coordinates_pairs)

In [ ]:
# Loop through all mapping methods and print their names and contents
for attr in dir(m):
    if not attr.startswith("_"):
        value = getattr(m, attr)
        print(f"{attr}:\n{value}\n")

In [99]:
# list all mapping methods
m.category_bases

['Kvalitet', 'Hälsa', 'Lönsamhet', 'Värdering', 'Kassaflöde']

In [102]:
m.cluster_columns

['long_trend_clusterRank',
 'ttm_momentum_clusterRank',
 'ttm_current_clusterRank']

In [8]:
m.cluster_config

{'long_trend': {'data_source': 'annual',
  'metric_type': 'trend',
  'description': 'Trend över 4 år',
  'col_name': 'Trend 4 år'},
 'ttm_momentum': {'data_source': 'quarterly',
  'metric_type': 'trend',
  'description': 'TTM-momentum, senaste 2 kvartalen',
  'col_name': 'TTM trend 2Q'},
 'ttm_current': {'data_source': 'quarterly',
  'metric_type': 'latest',
  'description': 'Senaste TTM-värden',
  'col_name': 'TTM senaste Q'}}

In [104]:
m.category_bases

['Kvalitet', 'Hälsa', 'Lönsamhet', 'Värdering', 'Kassaflöde']

In [105]:
# Get category rank column for Kvalitet in long_trend period
m.get_category_rank_column('Kvalitet', 'long_trend')
# Returns: 'Kvalitet_long_trend_catRank'

'Kvalitet_long_trend_catRank'

In [106]:
df_results.loc[selected_ticker, m.get_category_rank_column('Kvalitet', 'long_trend')]

np.float64(78.6)

In [107]:
# Get column display names
col_names = {period: m.get_cluster_col_name(period) for period in m.period_types}
# Result: {'long_trend': 'Trend 4 år', 'ttm_momentum': 'TTM trend 2Q', 'ttm_current': 'TTM senaste Q'}
col_names

{'long_trend': 'Trend 4 år',
 'ttm_momentum': 'TTM trend 2Q',
 'ttm_current': 'TTM senaste Q'}

In [108]:
def get_category_ranks_by_period(ticker, df, mappings):
    """
    Get category ranks for a stock across all periods and categories.
    
    Args:
        ticker: Stock ticker symbol (e.g., 'AAK')
        df: DataFrame with stock evaluation results (index is ticker)
        mappings: ConfigMappings instance
    
    Returns:
        DataFrame with:
        - Rows: category_base (Kvalitet, Hälsa, Lönsamhet, etc.) + Cluster rank row at the end
        - Columns: period_types (long_trend, ttm_momentum, ttm_current)
        - Values: Category ranks and cluster ranks from the evaluation results
    """
    # Create a dictionary to store the data
    data = {}
    
    # Iterate over periods (columns)
    for period in mappings.period_types:
        period_data = {}
        
        # Iterate over categories (rows)
        for category in mappings.category_bases:
            # Get the column name for this category and period
            col_name = mappings.get_category_rank_column(category, period)
            
            # Get the value from the dataframe
            try:
                value = df.loc[ticker, col_name]
                period_data[category] = value
            except KeyError:
                period_data[category] = None
        
        # Add cluster rank for this period
        cluster_col = mappings.cluster_mappings[period]
        try:
            cluster_value = df.loc[ticker, cluster_col]
            period_data['Agg. Rank'] = cluster_value
        except KeyError:
            period_data['Agg. Rank'] = None
        
        data[period] = period_data
    
    # Convert to DataFrame with categories as rows and periods as columns
    result_df = pd.DataFrame(data)

    # rename columns using mappings.period_descriptions
    col_names = {period: m.get_cluster_col_name(period) for period in m.period_types}
    result_df.rename(columns=col_names, inplace=True)

    return result_df

# Test the function
result = get_category_ranks_by_period(selected_ticker, df_results, m)
result


,Trend 4 år,TTM trend 2Q,TTM senaste Q
Kvalitet,78.6,50.0,64.3
Hälsa,100.0,100.0,100.0
Lönsamhet,100.0,57.1,85.7
Värdering,85.7,42.9,78.6
Kassaflöde,71.4,71.4,28.6
Agg. Rank,100.0,85.7,78.6


In [116]:
def get_ratio_ranks_by_period(ticker, ratio, df, mappings):
    """
    Get ratio ranks for a stock across all periods.
    
    Args:
        ticker: Stock ticker symbol (e.g., 'AAK')
        ratio: Ratio name (e.g., 'ROE', 'Total_Revenue')
        df: DataFrame with stock evaluation results (index is ticker)
        mappings: ConfigMappings instance
    
    Returns:
        DataFrame with:
        - One row: the ratio name
        - Columns: period_types (long_trend, ttm_momentum, ttm_current) with display names
        - Values: Ratio ranks for each period
    """
    # Create a dictionary to store the data
    data = {}
    
    # Iterate over periods
    for period in mappings.period_types:
        # Construct the column name for this ratio and period
        # Assuming pattern like: {ratio}_{period}_ratioRank
        col_name = f"{ratio}_{period}_ratioRank"
        
        # Get the value from the dataframe
        try:
            value = df.loc[ticker, col_name]
            data[period] = value
        except KeyError:
            data[period] = None
    
    # Convert to DataFrame with one row
    result_df = pd.DataFrame([data])
    result_df.index = [ratio]
    
    # Rename columns using display names from config
    col_names = {period: mappings.get_cluster_col_name(period) for period in mappings.period_types}
    result_df.rename(columns=col_names, inplace=True)
    
    return result_df

# Test the function
test_ratio = get_ratio_ranks_by_period(selected_ticker, 'ROE', df_results, m)
test_ratio

,Trend 4 år,TTM trend 2Q,TTM senaste Q
ROE,71.4,42.9,42.9


In [117]:
def get_ratio_values_by_period(ticker, ratio, df):
    """
    Get ratio values for a stock across annual and TTM periods.
    
    Args:
        ticker: Stock ticker symbol (e.g., 'AAK')
        ratio: Ratio/metric name (e.g., 'ROE', 'Total_Revenue')
        df: DataFrame with stock evaluation results (index is ticker)
    
    Returns:
        DataFrame with:
        - One row: the ticker
        - Columns: Last 4 annual values + Last 2 TTM values (6 columns total)
        - Values: Ratio values for each period
    """
    # Find all year columns for this ratio
    year_cols = [col for col in df.columns if col.startswith(ratio + '_year_')]
    year_cols = [col for col in year_cols if not pd.isna(df.loc[ticker, col])]
    year_cols_sorted = sorted(year_cols, key=lambda x: int(x.split('_')[-1]), reverse=False)
    year_cols_last4 = year_cols_sorted[-4:]
    
    # Find all quarter/TTM columns for this ratio
    # Looking for columns with pattern like: {ratio}_quarter_2... or {ratio}_ttm_...
    quarter_cols = [col for col in df.columns if '_quarter_' in col and ratio in col and '2' in col]
    quarter_cols = [col for col in quarter_cols if not pd.isna(df.loc[ticker, col])]
    
    # Sort quarters chronologically
    def quarter_sort_key(col):
        last_part = col.split('_')[-1]  # e.g., "2025Q1"
        if 'Q' in last_part:
            try:
                year_part = last_part.split('Q')[0]
                quarter_part = last_part.split('Q')[1]
                return int(year_part), int(quarter_part)
            except (ValueError, IndexError):
                return (0, 0)
        return (0, 0)
    
    quarter_cols_sorted = sorted(quarter_cols, key=quarter_sort_key, reverse=False)
    quarter_cols_last2 = quarter_cols_sorted[-2:]
    
    # Build the data dictionary
    data = {}
    
    # Add the 4 annual values
    for col in year_cols_last4:
        year = col.split('_')[-1]
        data[f"Year {year}"] = df.loc[ticker, col]
    
    # Add the 2 TTM values
    for col in quarter_cols_last2:
        quarter = col.split('_')[-1]  # e.g., "2025Q1"
        data[f"TTM {quarter}"] = df.loc[ticker, col]
    
    # Convert to DataFrame with one row
    result_df = pd.DataFrame([data])
    result_df.index = [ratio]
    
    return result_df

# Test the function
test_values = get_ratio_values_by_period(selected_ticker, 'ROE', df_results)
test_values

,Year 2021,Year 2022,Year 2023,Year 2024,TTM 2025Q1,TTM 2025Q2
ROE,0.121955,0.117717,0.17211,0.176034,0.183647,0.181464


In [118]:
get_ratio_values_by_period(selected_ticker, 'Soliditet', df_results)

,Year 2021,Year 2022,Year 2023,Year 2024,TTM 2025Q1,TTM 2025Q2
Soliditet,0.433741,0.442365,0.560276,0.584162,0.566501,0.607268


In [190]:
import plotly.graph_objects as go
import numpy as np

def plot_ratio_values(df,mappings):
    """
    Create a bar plot from ratio values dataframe.

    Args:
        df: pandas.DataFrame with 1 row and 6 columns (result from get_ratio_values_by_period).
            First 4 columns: annual values (royalblue).
            Last 2 columns: TTM values (gold).
        mappings: ConfigMappings instance (object providing ratio metadata and period mappings).

    Returns:
        plotly.graph_objects.Figure: Bar plot visualizing ratio values and trends.

    Error Handling:
        - If `df` contains missing or non-numeric values, bars will display 'nan' and trend lines may be omitted.
        - If `mappings.is_higher_better(ratio_name)` returns None, defaults to True.
        - Function assumes correct DataFrame shape; malformed input may result in plotting errors.
    """
    # Get the values and column names
    values = df.iloc[0].values.astype(float)
    columns = df.columns.tolist()
    ratio_name = df.index[0]

    # get higher_is_better info from mappings (default to True if None)
    hib = mappings.is_higher_better(ratio_name)
    higher_is_better = hib if hib is not None else True

    # Create colors: first 4 blue, last 2 gold
    colors = ['royalblue'] * 4 + ['gold'] * 2
    #colors = ['#D3D3D3'] * 4 + ['#4A90E2'] * 2
    trend_color_improving = '#70AD47'
    trend_color_deterioating = '#C5504E'
    trend_line_width = 6
    font_size = 18

    # Create patterns: 5th bar has '/' pattern
    patterns = [''] * 4 + ['\\'] + ['']
    
    # Create bar plot
    fig = go.Figure(data=[
        go.Bar(
            x=columns,
            y=[f"{v:.3f}" for v in values],#values,
            marker_color=colors,
            marker_pattern_shape=patterns,
            text=[f"{v:.2f}" for v in values],
            textposition='auto',
            textfont=dict(size=font_size),
            showlegend=False,
            name=ratio_name
        )
    ])
    
    # Add linear regression line for first 4 bars
    x_indices_1 = np.array([0, 1, 2, 3])
    y_values_1 = values[:4]
    coeffs_1 = np.polyfit(x_indices_1, y_values_1, 1)
    y_fit_1 = np.polyval(coeffs_1, x_indices_1)

    annual_diff =  values[3] - values[0]
    if (annual_diff > 0 and higher_is_better) or (annual_diff < 0 and not higher_is_better):
        annual_trend_color = trend_color_improving
    else:
        annual_trend_color = trend_color_deterioating
    
    fig.add_trace(go.Scatter(
        x=columns[:4],
        y=[f"{v:.3f}" for v in y_fit_1],#y_fit_1,
        mode='lines',
        name='Trend (4-year)',
        line=dict(color=annual_trend_color, width=trend_line_width, dash='dash'),
        showlegend=False
    ))

    # Add linear regression line for last 2 bars
    ttm_diff = values[5] - values[4]

    if (ttm_diff > 0 and higher_is_better) or (ttm_diff < 0 and not higher_is_better):
        ttm_trend_color = trend_color_improving
    else:
        ttm_trend_color = trend_color_deterioating

    x_indices_2 = np.array([0, 1])
    y_values_2 = values[4:6]
    coeffs_2 = np.polyfit(x_indices_2, y_values_2, 1)
    y_fit_2 = np.polyval(coeffs_2, x_indices_2)
    
    fig.add_trace(go.Scatter(
        x=columns[4:6],
        y=y_fit_2,
        mode='lines',
        name='Trend (TTM)',
        line=dict(color=ttm_trend_color, width=trend_line_width, dash='dash'),
        showlegend=False
    ))

    # Add annotation with the difference
    fig.add_annotation(
        x=columns[5],
        y=values[5],
        text=f"{ttm_diff:+.2g}",
        font=dict(size=font_size, color=ttm_trend_color),
        yshift=20,
        xshift=5,
        showarrow=False
    )
    
    # Add 'Data missing' annotations for each NaN value
    for i, v in enumerate(values):
        if pd.isna(v):
            fig.add_annotation(
                x=columns[i],
                y=0,
                text="Data missing",
                showarrow=False,
                font=dict(color="#b30000", size=13),
                bgcolor="#ffe5e5",
                bordercolor="#ffcccc",
                borderwidth=1,
                yshift=20
            )
    
    # Update layout
    fig.update_layout(
        height=400,
        margin=dict(l=10, r=10, t=40, b=10),
        hovermode='x unified',
        showlegend=False,
        xaxis=dict(fixedrange=True),
        yaxis=dict(fixedrange=True)
    )
    
    return fig

# Test the function
test_df = get_ratio_values_by_period(selected_ticker, 'PE_tal', df_results)
fig = plot_ratio_values(test_df,mappings=m)
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': [royalblue, royalblue, royalblue, royalblue,
                                   gold, gold],
                         'pattern': {'shape': ['', '', '', '', '\\', '']}},
              'name': 'PE_tal',
              'showlegend': False,
              'text': [33.09, 25.04, 19.18, 22.84, 19.40, 20.21],
              'textfont': {'size': 18},
              'textposition': 'auto',
              'type': 'bar',
              'x': [Year 2021, Year 2022, Year 2023, Year 2024, TTM 2025Q1, TTM
                    2025Q2],
              'y': [33.086, 25.041, 19.185, 22.843, 19.397, 20.212]},
             {'line': {'color': '#70AD47', 'dash': 'dash', 'width': 6},
              'mode': 'lines',
              'name': 'Trend (4-year)',
              'showlegend': False,
              'type': 'scatter',
              'x': [Year 2021, Year 2022, Year 2023, Year 2024],
              'y': [30.526, 26.868, 23.209, 19.551]},
             {'line': {'color': '#C5504E', 'dash': 'dash', 'width': 6},
              'mode': 'lines',
              'name': 'Trend (TTM)',
              'showlegend': False,
              'type': 'scatter',
              'x': [TTM 2025Q1, TTM 2025Q2],
              'y': {'bdata': 'uwRUk6tlM0BaXv2uRDY0QA==', 'dtype': 'f8'}}],
    'layout': {'annotations': [{'font': {'color': '#C5504E', 'size': 18},
                                'showarrow': False,
                                'text': '+0.81',
                                'x': 'TTM 2025Q2',
                                'xshift': 5,
                                'y': np.float64(20.21198552785641),
                                'yshift': 20}],
               'height': 400,
               'hovermode': 'x unified',
               'margin': {'b': 10, 'l': 10, 'r': 10, 't': 40},
               'showlegend': False,
               'template': '...',
               'xaxis': {'fixedrange': True},
               'yaxis': {'fixedrange': True}}
})

In [154]:
ratio_name = 'PB_tal'
higher_is_better = m.is_higher_better(ratio_name) or True
print(higher_is_better)

True


In [155]:
m.is_higher_better(ratio_name)

False